In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [2]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [3]:
input_folder = '../data/railways/Newark_LR/'
input_file = input_folder+'Passenger_Railroad_Lines_in_New_Jersey.shp'

output_folder = '../data/output_images/NewarkLR/'
output_csv = output_folder+'NEW.csv'

update_folder = output_folder+'/set_2/'

### Import Routes

In [3]:
routes = gpd.read_file(input_file)

routes = routes.to_crs({'init' :'epsg:4269'})

routes.head()

,OBJECTID,RAIL_LINE,SERVICE,Shape_Leng,DATE_STAMP,geometry
0,1,ATLANTIC CITY RAIL LINE,None,356957.019400,2016-08-30T00:00:00.000Z,LINESTRING (-74.44147959928382 39.363302944444...
1,2,BERGEN COUNTY LINE,HOBOKEN,155780.575084,2013-11-04T00:00:00.000Z,LINESTRING (-74.02804352228098 40.734841217805...
2,3,MAIN LINE,HOBOKEN,161721.051881,2013-11-04T00:00:00.000Z,LINESTRING (-74.1534482892289 41.1135474560739...
3,4,MEADOWLANDS RAIL LINE,None,56328.562168,2013-11-04T00:00:00.000Z,LINESTRING (-74.02804352228098 40.734841217805...
4,5,MONTCLAIR BOONTON LINE,NEW YORK CITY,328910.733006,2013-11-04T00:00:00.000Z,LINESTRING (-74.83535386691111 40.851441958709...


In [4]:
# Number of routes
routes.shape

(30, 6)

In [5]:
'''
Get all points
'''

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    try:
        route = routes.iloc[i]
        g = [m for m in route.geometry]
        for j in range(0,len(g)):
            sec_points = mapping(g[j])["coordinates"]
            for k in range(0,len(sec_points)):
                points.append(sec_points[k])
    except Exception as e:
        print("Skipped route number",i,'because',e)

print(len(points),'points')

Skipped route number 0 because 'LineString' object is not iterable
Skipped route number 1 because 'LineString' object is not iterable
Skipped route number 2 because 'LineString' object is not iterable
Skipped route number 3 because 'LineString' object is not iterable
Skipped route number 4 because 'LineString' object is not iterable
Skipped route number 5 because 'LineString' object is not iterable
Skipped route number 6 because 'LineString' object is not iterable
Skipped route number 7 because 'LineString' object is not iterable
Skipped route number 8 because 'LineString' object is not iterable
Skipped route number 9 because 'LineString' object is not iterable
Skipped route number 10 because 'LineString' object is not iterable
Skipped route number 11 because 'LineString' object is not iterable
Skipped route number 12 because 'LineString' object is not iterable
Skipped route number 13 because 'LineString' object is not iterable
Skipped route number 14 because 'LineString' object is not

In [ ]:
'''
Get points for 1 or more routes
'''

rows = [0]

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    if i in rows:
    #     print('Route',i)
        try:
            route = routes.iloc[i]
            g = [m for m in route.geometry]
    #         print('\t',len(g),'sections')
            for j in range(0,len(g)):
    #             print('   section',j)
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
    #                 print('      point',k)
                    points.append(sec_points[k])
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

In [ ]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

In [6]:
'''
Get subselection of points
'''

route_points = []
for point in range(0,len(points),28):
    tmp = points[point]
    route_points.append(tmp)

print(len(route_points))

48


In [7]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]


df.head()

,Name,Longitude,Latitude
0,-74.05845669376798_40.70884040173425,-74.058457,40.708840
1,-74.12620906945746_40.654272756355624,-74.126209,40.654273
2,-74.03693162147681_40.73212872881783,-74.036932,40.732129
3,-74.03701150336938_40.72429823532735,-74.037012,40.724298
4,-74.03434450827966_40.71743624880018,-74.034345,40.717436


In [8]:
'''
Export points
'''

df.to_csv(output_csv,header=True,index=None)

In [4]:
'''
Open csv
'''

df = pd.read_csv(output_csv)
df.head()

,Name,Longitude,Latitude,Catenary
0,-73.99942158108672_40.734131217013584,-73.999422,40.734131,0
1,-74.00713156551443_40.732835218887026,-74.007132,40.732835,0
2,-74.0085845620593_40.73108522271284,-74.008585,40.731085,0
3,-74.01026255940353_40.73120122234362,-74.010263,40.731201,0
4,-74.01245755256961_40.71078926264628,-74.012458,40.710789,0


In [5]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 13
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [29]:
'''
Get satellite preview for image
'''

row = 0

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [6]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 